In [17]:
from sklearn.datasets import make_classification
import torch
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        current_sample = self.data[idx, :]
        current_target = self.targets[idx]
        return {
            "sample": torch.tensor(current_sample, dtype = torch.float),
            "target": torch.tensor(current_target, dtype = torch.long)
        }

In [18]:
data, targets = make_classification(n_samples = 1000)

In [19]:
data.shape

(1000, 20)

In [20]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_targets, test_targets = train_test_split(data, 
                                                                      targets, 
                                                                      stratify = targets
                                                                     ) 

In [21]:
train_dataset = CustomDataset(train_data, train_targets)
test_dataset = CustomDataset(test_data, test_targets)

In [29]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size = 4, num_workers = 0)

test_dataloader  = DataLoader(test_dataset, batch_size = 4, num_workers = 0)

In [28]:
print(train_data.shape)
print(test_data.shape)

(750, 20)
(250, 20)


In [24]:
model = lambda x, w, b: torch.matmul(x, w) + b

In [33]:
W = torch.randn(20,1, requires_grad = True)
b = torch.randn(1, requires_grad = True)
learning_rate = 0.001

for epoch in range(10):
    epoch_loss = 0
    counter = 0
    for data in train_dataloader:
        xtrain = data['sample']
        ytrain = data['target']
        
        output = model(xtrain,W, b)
        loss = torch.mean((ytrain.view(-1) - output.view(-1))**2)
        epoch_loss = epoch_loss + loss.item()
        loss.backward()
        
        with torch.no_grad():
            W = W - learning_rate * W.grad
            b = b - learning_rate * b.grad
            
        W.requires_grad_(True)
        b.requires_grad_(True)
        counter += 1
    print(epoch, epoch_loss/counter)

0 18.957969337701797
1 7.203261010190274
2 3.24255100883385
3 1.5516151989354414
4 0.7853904986318122
5 0.427766730860272
6 0.2572088811764533
7 0.17441914316038898
8 0.13361685049343616
9 0.113232208950881


In [36]:
outputs = []
labels = []

with torch.no_grad():
    for data in test_dataloader:
        xtest = data['sample']
        ytest = data['target']
        
        output = model(xtest, W, b)
        labels.append(ytest)
        outputs.append(output)
        

In [37]:
outputs

[tensor([[0.8477],
         [0.2488],
         [1.1314],
         [0.3537]]),
 tensor([[ 0.4991],
         [ 0.7572],
         [-0.3234],
         [ 1.0096]]),
 tensor([[ 0.6324],
         [ 0.8027],
         [ 0.4970],
         [-0.2572]]),
 tensor([[0.0429],
         [0.2980],
         [0.9922],
         [1.0504]]),
 tensor([[0.0629],
         [0.2364],
         [1.5257],
         [0.6424]]),
 tensor([[0.6038],
         [1.0340],
         [0.8930],
         [0.7355]]),
 tensor([[0.2931],
         [0.8750],
         [1.1134],
         [0.3893]]),
 tensor([[1.0047],
         [0.7066],
         [0.7379],
         [0.9271]]),
 tensor([[ 0.9619],
         [ 0.7083],
         [-0.0244],
         [ 0.0498]]),
 tensor([[-0.1026],
         [-0.3664],
         [ 0.6292],
         [-0.1837]]),
 tensor([[ 0.0389],
         [-0.0076],
         [-0.5019],
         [ 0.4271]]),
 tensor([[-0.1978],
         [-0.0076],
         [ 0.5289],
         [ 0.4367]]),
 tensor([[0.3879],
         [0.8487],
  

In [38]:
torch.cat(outputs).view(-1)

tensor([ 0.8477,  0.2488,  1.1314,  0.3537,  0.4991,  0.7572, -0.3234,  1.0096,
         0.6324,  0.8027,  0.4970, -0.2572,  0.0429,  0.2980,  0.9922,  1.0504,
         0.0629,  0.2364,  1.5257,  0.6424,  0.6038,  1.0340,  0.8930,  0.7355,
         0.2931,  0.8750,  1.1134,  0.3893,  1.0047,  0.7066,  0.7379,  0.9271,
         0.9619,  0.7083, -0.0244,  0.0498, -0.1026, -0.3664,  0.6292, -0.1837,
         0.0389, -0.0076, -0.5019,  0.4271, -0.1978, -0.0076,  0.5289,  0.4367,
         0.3879,  0.8487,  0.8176,  0.8197, -0.1123, -0.6107,  0.7654, -0.2097,
         1.1200,  0.8526,  0.6533,  0.3017,  0.5405,  0.2026,  0.5159,  0.5910,
         0.7913,  0.1697,  0.9694,  0.4555, -0.3023,  0.8177,  0.9059, -0.1307,
         0.5781,  1.0958,  0.5068,  0.9393,  1.0495,  0.7333, -0.4426,  0.3623,
         0.1394,  0.2017,  0.4130,  0.7007,  0.3819,  0.5215,  0.9801, -0.0898,
        -0.3120,  1.0661, -0.0188,  0.0726,  0.4514,  0.0485,  0.5315,  1.0456,
         0.8476,  0.3676,  0.5048,  0.26

In [40]:
from sklearn import metrics
metrics.roc_auc_score(torch.cat(labels).view(-1), torch.cat(outputs).view(-1))

0.94496